# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [16]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
from sklearn.model_selection import BaseCrossValidator, ParameterGrid, ParameterSampler
import torch
import pickle
import shutil
import shap
from sklearn.model_selection import RepeatedStratifiedKFold
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from sklearn.model_selection import train_test_split
import numpy as np
from pytorch_tabular.utils import make_mixed_dataset, print_metrics
from pytorch_tabular import available_models
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, GANDALFConfig, TabNetModelConfig, FTTransformerConfig, DANetConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.tabular_model_tuner import TabularModelTuner
from torchmetrics.functional.regression import mean_absolute_error, pearson_corrcoef
from pytorch_tabular import MODEL_SWEEP_PRESETS
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.utils.hash import dict_hash
from src.pt.hyper_opt import train_hyper_opt
import pathlib
from tqdm import tqdm
import distinctipy
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
from regression_bias_corrector import LinearBiasCorrector
import optuna
from sklearn.preprocessing import LabelEncoder
from plottable import ColumnDefinition, Table
from plottable.plots import bar
from plottable.cmap import normed_cmap, centered_cmap
import matplotlib.lines as mlines
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats
from functools import reduce
import shutil
import os
from pypdf import PdfReader
import re
import logging
from slugify import slugify

logging.getLogger("pypdf").setLevel(logging.ERROR)


def make_rgb_transparent(rgb, bg_rgb, alpha):
    return [alpha * c1 + (1 - alpha) * c2 for (c1, c2) in zip(rgb, bg_rgb)]

# Load components

In [ ]:
dir_root = f"E:/Git/MillenniumAge"

feat_trgt = 'Возраст'

components = {
    'Оценка состава тела, женщины': {
        'name': 'Оценка состава тела',
        'path': f"{dir_root}/data/Оценка состава тела/females",
        'bkg_count': 88,
        'likelihood': 0.35
    },
    'Оценка состава тела, мужчины': {
        'name': 'Оценка состава тела',
        'path': f"{dir_root}/data/Оценка состава тела/males",
        'bkg_count': 30,
        'likelihood': 0.65
    },
    
    'Электрокардиограмма, все': {
        'name': 'Электрокардиограмма',
        'path': f"{dir_root}/data/Электрокардиограмма/all",
        'bkg_count': 130,
        'likelihood': 0.4
    },
    'Электрокардиограмма, старше 15': {
        'name': 'Электрокардиограмма',
        'path': f"{dir_root}/data/Электрокардиограмма/over15",
        'bkg_count': 120,
        'likelihood': 0.25
    },
    'Электрокардиограмма, младше 15': {
        'name': 'Электрокардиограмма',
        'path': f"{dir_root}/data/Электрокардиограмма/under15",
        'bkg_count': 20,
        'likelihood': 0.25
    },
    
    'Гематологические исследования, все': {
        'name': 'Гематологические исследования',
        'path': f"{dir_root}/data/Гематологические исследования/all",
        'bkg_count': 300,
        'likelihood': 0.3
    },
    'Гематологические исследования, старше 15': {
        'name': 'Гематологические исследования',
        'path': f"{dir_root}/data/Гематологические исследования/over15",
        'bkg_count': 250,
        'likelihood': 0.17
    },
    'Гематологические исследования, младше 15': {
        'name': 'Гематологические исследования',
        'path': f"{dir_root}/data/Гематологические исследования/under15",
        'bkg_count': 50,
        'likelihood': 0.30
    },
    
    'Биохимические исследования 7, все': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/7/all",
        'bkg_count': 160,
        'likelihood': 0.7
    },
    'Биохимические исследования 7, старше 15': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/7/over15",
        'bkg_count': 150,
        'likelihood': 0.55
    },
    'Биохимические исследования 7, младше 15': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/7/under15",
        'bkg_count': 24,
        'likelihood': 0.8
    },
    'Биохимические исследования 9, все': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/9/all",
        'bkg_count': 150,
        'likelihood': 0.7
    },
    'Биохимические исследования 9, старше 15': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/9/over15",
        'bkg_count': 130,
        'likelihood': 0.55
    },
    'Биохимические исследования 9, младше 15': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/9/under15",
        'bkg_count': 20,
        'likelihood': 0.8
    },
    'Биохимические исследования 12': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/12",
        'bkg_count': 100,
        'likelihood': 0.6
    },
    'Биохимические исследования 23': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/23",
        'bkg_count': 100,
        'likelihood': 0.5
    },
    'Биохимические исследования 25, женщины': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/25/females",
        'bkg_count': 50,
        'likelihood': 0.58
    },
    'Биохимические исследования 25, мужчины': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/25/males",
        'bkg_count': 20,
        'likelihood': 0.37
    },
    
    'Половые гормоны 6, женщины': {
        'name': 'Половые гормоны',
        'path': f"{dir_root}/data/Половые гормоны/females/6",
        'bkg_count': 62,
        'likelihood': 0.85
    },
    'Половые гормоны 8, женщины': {
        'name': 'Половые гормоны',
        'path': f"{dir_root}/data/Половые гормоны/females/8",
        'bkg_count': 45,
        'likelihood': 0.75
    },
    'Половые гормоны 6, мужчины': {
        'name': 'Половые гормоны',
        'path': f"{dir_root}/data/Половые гормоны/males/6",
        'bkg_count': 36,
        'likelihood': 0.55
    },
    'Половые гормоны 10, мужчины': {
        'name': 'Половые гормоны',
        'path': f"{dir_root}/data/Половые гормоны/males/10",
        'bkg_count': 20,
        'likelihood': 0.7
    },
    
    'Гормоны 3, все': {
        'name': 'Гормоны',
        'path': f"{dir_root}/data/Гормоны/3/all",
        'bkg_count': 200,
        'likelihood': 0.25
    },
    'Гормоны 3, старше 18': {
        'name': 'Гормоны',
        'path': f"{dir_root}/data/Гормоны/3/over18",
        'bkg_count': 200,
        'likelihood': 0.1
    },
    'Гормоны 3, младше 18': {
        'name': 'Гормоны',
        'path': f"{dir_root}/data/Гормоны/3/under18",
        'bkg_count': 14,
        'likelihood': 0.35
    },
    'Гормоны 5': {
        'name': 'Гормоны',
        'path': f"{dir_root}/data/Гормоны/5",
        'bkg_count': 140,
        'likelihood': 0.25
    },
    'Гормоны 6': {
        'name': 'Гормоны',
        'path': f"{dir_root}/data/Гормоны/6",
        'bkg_count': 120,
        'likelihood': 0.25
    },
}

feats_all = ['Пол']
feats_pred_all = []
feats_input_all = []
feats_aux = []
for comp in components:
    components[comp]['data'] = pd.read_excel(f"{components[comp]['path']}/data.xlsx", index_col=0)
    components[comp]['feats'] = pd.read_excel(f"{components[comp]['path']}/feats.xlsx", index_col=0)
    components[comp]['results'] = pd.read_excel(f"{components[comp]['path']}/model/df.xlsx", index_col=0)
    components[comp]['metrics'] = pd.read_excel(f"{components[comp]['path']}/model/metrics.xlsx", index_col=0)
    components[comp]['model'] = TabularModel.load_model(f"{components[comp]['path']}/model")
    components[comp]['corrector'] = LinearBiasCorrector()
    comp_results = components[comp]['results']
    components[comp]['corrector'].fit(comp_results.loc[comp_results['Group'] == 'Train', feat_trgt].values, comp_results.loc[comp_results['Group'] == 'Train', 'Prediction'].values)
    res_cols = ['Group', 'Prediction', 'Error', 'Prediction Unbiased', 'Error Unbiased']
    components[comp]['data'].loc[components[comp]['data'].index, res_cols] = comp_results.loc[components[comp]['data'].index, res_cols]
    components[comp]['data_shap'] = components[comp]['data'].copy()
    
    feats = components[comp]['feats'].index.values
    feats = feats[feats != feat_trgt]
    feats_all += list(feats)
    feats_pred_all += [f"Предсказание {components[comp]['name']}", f"Возрастная Акселерация {components[comp]['name']}", f"Модель {components[comp]['name']}",]
    feats_all += [f"Модель {comp}", f"Предсказание {comp}", f"Возрастная Акселерация {comp}"]
    feats_aux += [f"Модель {comp}", f"Предсказание {comp}", f"Возрастная Акселерация {comp}"]
    feats_input_all += list(feats)
    
    components[comp]['feats_corr'] = pd.DataFrame(index=feats, columns=['Correlation'])
    for f in feats:
        components[comp]['feats_corr'].at[f, 'Correlation'], _ = scipy.stats.pearsonr(components[comp]['data'].loc[:, f].values, components[comp]['data'].loc[:, feat_trgt].values)

feats_all = list(dict.fromkeys(feats_all)) + list(dict.fromkeys(feats_pred_all))
feats_input_all = list(dict.fromkeys(feats_input_all))
feats_aux = list(dict.fromkeys(feats_aux))

for comp in components:
    print(f"{comp}: {components[comp]['data'].shape[0]}")

# Check local explainability

In [12]:
path = 'E:/YandexDisk/Work/bbd/millennium/Результаты чекап'

sample_folder = 'Садчикова_raw' # '9077404592_Михайлова_Ольга_Игоревна_2024_raw'

samples = pd.read_pickle(f"{path}/{sample_folder}/data.pkl")
samples_imputed = pd.read_pickle(f"{path}/{sample_folder}/is_imputed.pkl")

trgt_id = samples.index[0]

In [ ]:
local_exlp = {}

n_pos = samples.at[trgt_id, "Число моделей c положительной аккселерацией"]
n_neg = samples.at[trgt_id, "Число моделей c отрицательной аккселерацией"]
trgt_age = samples.at[trgt_id, feat_trgt]

components_exist = []
for comp in components:
    if samples.at[trgt_id, f"Модель {comp}"] == True:
        components_exist.append(comp)
        
comp = 'Электрокардиограмма, старше 15'

feats = components[comp]['feats'].index.values
feats_corr = components[comp]['feats_corr']
metrics = components[comp]['metrics']
data_shap = components[comp]['data_shap']

def predict_func(X):
    X_df = pd.DataFrame(data=X, columns=feats)
    y = components[comp]['model'].predict(X_df)[f'{feat_trgt}_prediction'].values
    y = components[comp]['corrector'].predict(y)
    return y

mae = metrics.at['Test', 'mean_absolute_error_unbiased']
# rho = metrics.at['Test', 'pearson_corrcoef_unbiased'] * components[comp]['likelihood']
rho = metrics.at['Test', 'pearson_corrcoef_unbiased'] * feats_corr['Correlation'].abs().max()

bkg_count = components[comp]['bkg_count']

trgt_pred_raw = samples.at[trgt_id, f"Предсказание {comp}"]

data_closest_step_1 = data_shap.loc[data_shap['Error Unbiased'].abs() < mae * rho, :]
data_closest_step_2 = data_closest_step_1.iloc[(data_closest_step_1['Prediction Unbiased'] - trgt_age).abs().argsort()[:bkg_count]]

pathlib.Path(f"{path}/{sample_folder}/{comp}").mkdir(parents=True, exist_ok=True)

for f in feats:
    

